In [ ]:
import pandas as pd
import numpy as np
from glob import glob
import holoviews as hv
import panel as pn
import panel.widgets as pnw
hv.extension("bokeh")

def read_fukushima_data(file_path):
    return pd.read_csv(file_path, 
                 skiprows=[1,2],
                 index_col=0, 
                 parse_dates=True,
                 na_values=["ND","-"],
                 encoding="utf-8")

file_list = glob("fukushima_data/201[34]/*.csv")
df_list = [read_fukushima_data(f) for f in file_list]

common_columns = df_list[0].columns
df_dict = {}
for d in df_list:
    d.index.name = "Date"
    df_dict[str(d.index[0].year)] = d
    common_columns = common_columns.intersection(d.columns)

years = sorted(df_dict.keys())


columns = sorted(common_columns)
discrete = [x for x in columns if df_list[0][x].dtype == object]
continuous = [x for x in columns if x not in discrete]
quantileable = [x for x in continuous if len(df_list[0][x].unique()) > 20]

x = "Date"
y = pnw.CheckBoxGroup(name="Y-Axis", value=[quantileable[0]], options=quantileable)
year = pnw.CheckBoxGroup(name="Year", value=[years[0]], options=years, width=70)

@pn.depends(year.param.value, y.param.value)
def create_figure(year, y):
    if (len(year) == 0) or (len(y) == 0):
            return
    curve_list = []
    for yr in year:
        curve_list += [hv.Curve(df_dict[yr], [x, y]) for y in y]
    return hv.Overlay(curve_list).opts(width=600, height=450)
    
#widgets = pn.WidgetBox(year, y)

pn.Row(pn.Column("### Year:", year), pn.Column("### Y-Axis:", y), create_figure)